In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim 
from torchvision import datasets, transforms

In [2]:
import torchvision
print(torchvision.__version__)

0.24.1


In [3]:
datasets

<module 'torchvision.datasets' from '/Users/baijuyadav/Desktop/deeplearningfromscratch/deeplearningscratch/lib/python3.12/site-packages/torchvision/datasets/__init__.py'>

In [5]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# =====================
# Model
# =====================
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.dropout = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(self.dropout(x), 2)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

# =====================
# Setup
# =====================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# =====================
# Training
# =====================
model.train()
for epoch in range(1):
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()

# =====================
# Evaluation
# =====================
model.eval()
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)

        test_loss += nn.functional.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()

test_loss /= len(test_loader.dataset)

print(f"Test Loss: {test_loss:.4f}")
print(f"Accuracy: {correct}/{len(test_loader.dataset)}")


Test Loss: 0.0418
Accuracy: 9865/10000


In [8]:
import ssl
print(ssl.get_default_verify_paths())


DefaultVerifyPaths(cafile=None, capath=None, openssl_cafile_env='SSL_CERT_FILE', openssl_cafile='/Library/Frameworks/Python.framework/Versions/3.12/etc/openssl/cert.pem', openssl_capath_env='SSL_CERT_DIR', openssl_capath='/Library/Frameworks/Python.framework/Versions/3.12/etc/openssl/certs')


In [6]:
# inference code 
from PIL import Image
import torch
from torchvision import transforms

# Same transform as training
infer_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

def predict_digit(image_path, model, device):
    model.eval()

    image = Image.open(image_path)
    image = infer_transform(image)
    image = image.unsqueeze(0).to(device)  # add batch dimension

    with torch.no_grad():
        output = model(image)
        prediction = output.argmax(dim=1).item()

    return prediction


In [13]:
digit = predict_digit("5.png", model, device)
print("Predicted digit:", digit)


Predicted digit: 3
